# Análisis y visualización de datos abiertos con python


El objetivo de este tutorial es analizar dos conjuntos de datos abiertos de la Ciudad de México (Carpetas de investigación PGJ y Víctimas en carpetas de investigación PGJ) usando las herramientas del lenguaje de programación python. El participante aprenderá a usar las biblioteca pandas y otras bibliotecas, para cargar, estructurar, obtener estadísticas básicas y graficar los datos. Además, el participante aplicará metodologías y criterios de análisis y visualización de datos, que le permitan generar conclusiones sobre los conjuntos de datos analizados.


1. [Introducción a python](./CP-Introduccion.ipynb)
2. [Exploración de datos](./CP-Exploracion.ipynb)
3. __Limpieza de datos__
    1. Pasos de un análisis de datos
    2. Operaciones comunes de limpieza
        * Quitar columnas (drop, select)
        * Cambiar tipos de datos (datetime)
        * Modificar campos de texto (replace, title, unidecode)
        * Datos faltantes (fillna)
        * Eliminar datos fuera de rango (map, replace)
    5. Guardar datos
        * Excel y csv
        * pickle
4. [Gráficación básica](./CP-AnalisisGraficas.ipynb)
5. [Análisis de datos](./CP-AnalisisGraficas.ipynb)
6. Extras


<a id='Obtencion'></a>

<a id='Obtencion'></a>
## 3. Limpieza de datos

### 3.1 Pasos de un análisis de datos

Un análisis de datos es un proceso que lleva varios pasos. 

En primer lugar es importante entender los datos, es decir saber su origen y qué representan. Este paso es fundamental, ya que de poco sirve hacer un análisis complejo que no tenga utilidad con respecto a los datos que estamos análizando.

Una vez que se entiende de que tratan los datos se necesita determinar las características del conjunto de datos. Esto generalmente implica saber cuántos son, qué tipos, la distribución de los datos, obtener medidas estadísticas básicas, etc.

Después, se lleva a cabo una limpieza. Este proceso es generalmente el que toma más tiempo. Este paso es iterativo, conforme vamos conociendo más acerca de los datos, puede ser necesario hacer nuevas limpiezas o modificaciones. Para evitar errores es importante guardar los datos originales y documentar todas las transformaciones que se vayan haciendo a los datos y las razones de tales cambios.

El análisis de datos puede conllevar varias actividades como transformar los datos, resumirmos, unirlos con otros conjuntos de datos y finalmente visualizarlos. 

In [1]:
from os import chdir, getcwd
chdir(getcwd())

In [2]:
import pandas as pd
from numpy import nan

df_vic = pd.read_csv('data-raw/denuncias-victimas-pgj.csv')

### 3.4 Limpieza de datos

Generalmente, los conjuntos de datos contienen errores o incluyen datos que no son utiles para los análisis. Durante la limpieza de datos se detectan y corrigen datos incorrectos. Además, en este paso se quitan los datos que no se usaran en el análisis. 

En este caso haremos una serie de correciones:
* Quitar columnas
* Cambiar tipos de datos
* Modificar campos de texto
* Eliminar datos fuera de rango
* Datos faltantes
* Guardar datos

#### Quitar columnas

Podemos quitar las columnas usando el comando _.drop()_. En este caso quitamos las columas que tienen información redundante (año, mes, coordenadas) o que no nos interesan en este momento (agencia, unidad_investigacion, colonia, calle). Para quitar columnas se debe de agregar el parametro _axis=1_. Si lo que queremos es quitar filas, se puede hacer de una manera similar, poniendo los nombres de las filas entre corchetes y usando el paramentro _axis=0_. 

Para guardar este paso de limpieza es muy importante rescribir los valores de la variable.

In [3]:
df_vic = df_vic.drop(['idCarpeta', 'Año_hecho', 'Mes_hecho', 'ClasificacionDelito', 'lon', 'lat'], axis=1)
df_vic.columns = ['delito', 'categoria_delito', 'fecha_hechos', 
                  'Sexo', 'Edad', 'TipoPersona', 'CalidadJuridica', 'Geopoint']
df_vic.head()

,delito,categoria_delito,fecha_hechos,Sexo,Edad,TipoPersona,CalidadJuridica,Geopoint
0,NARCOMENUDEO POSESION SIMPLE,DELITO DE BAJO IMPACTO,2019-05-01 16:30:00,NaN,NaN,MORAL,VICTIMA,"19.3283464986,-99.0407749002"
1,ROBO A NEGOCIO SIN VIOLENCIA,DELITO DE BAJO IMPACTO,2019-03-28 10:00:00,Masculino,NaN,FISICA,VICTIMA Y DENUNCIANTE,"19.4020752972,-99.1525710006"
2,ROBO A NEGOCIO SIN VIOLENCIA,DELITO DE BAJO IMPACTO,2019-03-28 10:00:00,NaN,NaN,MORAL,VICTIMA,"19.4020752972,-99.1525710006"
3,ROBO A NEGOCIO SIN VIOLENCIA,DELITO DE BAJO IMPACTO,2019-05-01 13:30:00,NaN,NaN,MORAL,VICTIMA,"19.4349789028,-99.1470464003"
4,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,2019-04-30 18:30:00,Femenino,36.0,FISICA,VICTIMA Y DENUNCIANTE,"19.2571868053,-99.0817841843"


#### Cambiar tipos de datos

Como ya vimos exiten varios tipos de datos. En partícular en este conjunto de datos, tenemos dos columnas de fechas, y si revisamos a que tipo de dato pertenecen usando el comando _dtype_ ,podemos observar que pandas las identificó como texto, lo cual dificulta trabajar con ellas. Si cambiamos estas columnas a tipo _datetime_, podemos usar las operaciones definidas para este tipo de datos. Haremos lo anterior empleando el comando _to_datetime_. Debido a que nos interesa guardar la transformación, es importante recordar hacer la asignación con _=_.

In [4]:
df_vic['fecha_hechos'] = pd.to_datetime(df_vic['fecha_hechos'])

Usando este tipo de dato ahora es posible seleccionar por año, mes, día, hora y minuto usando los comandos:

* columna.dt.year
* columna.dt.month
* columna.dt.day
* columna.dt.hour
* columna.dt.minute

In [5]:
df_vic['fecha_hechos'].dt.hour

0         16.0
1         10.0
2         10.0
3         13.0
4         18.0
          ... 
145865     3.0
145866     4.0
145867     5.0
145868     4.0
145869     7.0
Name: fecha_hechos, Length: 145870, dtype: float64

Esto nos permite ver los delitos cometidos después de las 8pm. Para lograrlo usaremos _.dt.hour_ y el subsetting que ya hemos visto con anterioridad.

In [6]:
df_vic[df_vic['fecha_hechos'].dt.hour>=20]

,delito,categoria_delito,fecha_hechos,Sexo,Edad,TipoPersona,CalidadJuridica,Geopoint
13,ROBO A TRANSEUNTE EN VIA PUBLICA SIN VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,2019-04-24 20:20:00,Masculino,26.0,FISICA,VICTIMA Y DENUNCIANTE,"19.4201584349,-99.0700393867"
26,ABUSO SEXUAL,DELITO DE BAJO IMPACTO,2019-04-30 22:00:00,Femenino,20.0,FISICA,VICTIMA Y DENUNCIANTE,"19.3240315026,-99.1169818999"
27,SUSTRACCION DE MENORES,DELITO DE BAJO IMPACTO,2019-04-30 21:30:00,Femenino,14.0,FISICA,VICTIMA,"19.4027156981,-99.0793629997"
34,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,2019-04-30 23:00:00,Femenino,35.0,FISICA,VICTIMA Y DENUNCIANTE,"19.377112689,-99.053766085"
41,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,2019-04-30 22:00:00,Femenino,NaN,FISICA,VICTIMA,NaN
...,...,...,...,...,...,...,...,...
145856,TENTATIVA DE VIOLACION,DELITO DE BAJO IMPACTO,2019-09-19 22:20:00,Femenino,37.0,FISICA,VICTIMA Y DENUNCIANTE,"19.3563799373,-99.2877924663"
145857,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,2019-09-19 22:50:00,Masculino,23.0,FISICA,VICTIMA Y DENUNCIANTE,"19.4861320884,-99.0851386999"
145859,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,2019-09-19 22:20:00,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,"19.3281515772,-99.1480473403"
145860,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,2019-09-19 22:50:00,Masculino,51.0,FISICA,VICTIMA Y DENUNCIANTE,"19.4848701405,-99.0844488098"


#### Modificar campos de texto

Cuando todas las columnas de texto están con mayúsculas, como en el grupo de datos con el que estamos trabajando, puede dificultarse la lectura. En este caso, vamos a cambiar el texto de mayúsculas de los delitos usando el comando _.capitalize()_, es decir este comando dejará la primera letra en mayúscula y todas las demás en minúscula. En el caso de las alcaldías y fiscalias usaremos _.title()_, con el fin de que la primera letra de todas las palabras sea mayúscula.

In [7]:
df_vic['delito'] =           df_vic['delito'].str.capitalize()
df_vic['categoria_delito'] = df_vic['categoria_delito'].str.capitalize()
df_vic['TipoPersona'] =      df_vic['TipoPersona'].str.capitalize()
df_vic['CalidadJuridica'] =  df_vic['CalidadJuridica'].str.capitalize()
df_vic.tail()

,delito,categoria_delito,fecha_hechos,Sexo,Edad,TipoPersona,CalidadJuridica,Geopoint
145865,Lesiones culposas por caida de vehículo en mov...,Delito de bajo impacto,2019-09-20 03:30:00,Masculino,42.0,Fisica,Lesionado,"19.3443909603,-99.0279509938"
145866,Robo a transeunte en via publica con violencia,Robo a transeunte en vía pública con y sin vio...,2019-09-20 04:30:00,Masculino,45.0,Fisica,Victima y denunciante,"19.3061992087,-98.9656925188"
145867,Perdida de la vida por suicidio,Hecho no delictivo,2019-09-20 05:00:00,Masculino,33.0,Fisica,Cadaver,"19.4102934856,-99.1622807104"
145868,Homicidio por arma blanca,Homicidio doloso,2019-09-20 04:00:00,Masculino,27.0,Fisica,Cadaver,"19.3223447722,-99.0427655635"
145869,Lesiones culposas,Delito de bajo impacto,2019-09-18 07:20:00,Femenino,5.0,Fisica,Lesionado,NaN


#### Datos faltantes

Si revisamos los datos podemos ver que hay varios casos donde nos faltan datos, en especial en "Sexo" y "Edad".

In [8]:
df_vic.count()

delito              145870
categoria_delito    145870
fecha_hechos        145743
Sexo                122063
Edad                100547
TipoPersona         144444
CalidadJuridica     145870
Geopoint            140143
dtype: int64

En el caso de "Sexo" podemos ver que en algunos casos se reporta que este "No se especifica" mientras que en otros se deja vacio, es decir, con NaN.

In [9]:
df_vic['Sexo'].value_counts(dropna=False)

Masculino           68288
Femenino            53770
NaN                 23807
No se especifica        5
Name: Sexo, dtype: int64

En este caso vamos a remplazar todos los "NaN" por "No se especifica" usando el comando _.fillna()_

In [10]:
df_vic['Sexo'] = df_vic['Sexo'].fillna("No se especifica")
df_vic['Sexo'].value_counts(dropna=False)

Masculino           68288
Femenino            53770
No se especifica    23812
Name: Sexo, dtype: int64

#### Datos fuera de rango

__Fecha__

En el caso de este conjunto de datos tenemos las carpetas que se crearon entre 2016 y 2019. Sin embargo, esto no significa que todos los delitos hayan sido cometidos durante este periodo de tiempo. Para visualizarlo vamos a contar cuántas carpetas hay por año usando _.dt.year_ y _.value_counts()_.

In [11]:
df_vic['fecha_hechos'].dt.year.value_counts()

2019.0    134611
2018.0      8807
2017.0       971
2016.0       416
2015.0       224
2014.0       214
2013.0        96
2012.0        68
2011.0        60
2010.0        56
2009.0        46
2008.0        42
2001.0        19
2007.0        16
2006.0        15
2005.0        11
2004.0        10
1998.0         7
2002.0         6
2000.0         6
2003.0         5
1999.0         4
1972.0         3
1994.0         3
1983.0         3
1968.0         2
1974.0         2
1989.0         2
1995.0         2
1996.0         2
1997.0         2
1991.0         1
1990.0         1
1992.0         1
1987.0         1
1985.0         1
1993.0         1
1981.0         1
1980.0         1
1973.0         1
1971.0         1
1969.0         1
1964.0         1
Name: fecha_hechos, dtype: int64

En este caso podemos ver que hay carpetas sobre delitos cometidos en 1964. Para nuestro análisis vamos a quitar todas las carpetas que sean sobre hechos anteriores al 2016. Es decir, dejaremos unicamente las carpetas cuyo año sea mayor o igual a 2016.

In [12]:
df_vic = df_vic[df_vic['fecha_hechos'].dt.year>=2016]

__Edad__

En el caso de la edad podemos recordar que hay una persona que tiene una edad de 258 años. Probablemente este sea un error de captura, ya que es la única persona reportada de mas de cién años. 
En este caso vamos a poner un críterio para manejar problemas similares a futuro, si la persona tiene mas de cién años remplazaremos su edad por nan. 
Para lograr esto seleccionaremos todas las celdas de personas de mas de cién años que están en la columna Edad con _.loc[]_ y remplazaremos estos valores por NaN. 
Esto debé de cambiar la edad máxima de nuestros datos.

In [13]:
df_vic.loc[df_vic['Edad']>=100,'Edad'] = nan
df_vic['Edad'].max()

99.0

Otra cuestión que cabe destacar es que hay una gran cantidad de víctimas con edad de cero años. Este es un problema de la captura de los datos, ya que muchas veces, cuando no se conoce la edad de la víctima se pone un cero, lo cual se puede confundir con casos donde la víctima es un menor de edad. Sin embargo, en este caso es imposible corregir este error, por lo que dejaremos los datos como están, pero tendremos cuidado de esto durante el análisis.

In [14]:
df_vic[df_vic['Edad']==0]

,delito,categoria_delito,fecha_hechos,Sexo,Edad,TipoPersona,CalidadJuridica,Geopoint
1858,Narcomenudeo posesion simple,Delito de bajo impacto,2019-05-10 22:45:00,No se especifica,0.0,Moral,Victima,"19.3561934036,-99.3005525056"
3280,Violencia familiar,Delito de bajo impacto,2019-07-30 12:00:00,Masculino,0.0,Fisica,Victima niño,"19.4918033156,-99.1217867298"
3870,Aborto,Delito de bajo impacto,2019-09-23 20:30:00,No se especifica,0.0,Moral,Cadaver,"19.552463053,-99.14245583"
4383,Lesiones culposas por caida,Delito de bajo impacto,2019-09-27 01:25:00,Masculino,0.0,Fisica,Victima,NaN
4522,Violencia familiar,Delito de bajo impacto,2019-09-24 17:00:00,Femenino,0.0,Fisica,Victima,"19.447493298,-99.1488042999"
...,...,...,...,...,...,...,...,...
141086,Feminicidio,Homicidio doloso,2019-03-20 15:15:00,Femenino,0.0,Fisica,Cadaver,"19.4690326848,-99.1543045098"
141985,Corrupcion de menores,Delito de bajo impacto,2019-03-25 12:30:00,Femenino,0.0,Fisica,Victima,"19.2810964993,-99.2028764017"
142508,Denuncia de hechos,Hecho no delictivo,2019-03-27 19:11:00,No se especifica,0.0,Fisica,Victima,NaN
144459,Aborto,Delito de bajo impacto,2019-09-11 10:00:00,No se especifica,0.0,Moral,Cadaver,NaN


__Categorias y delitos__

En el pandas_profiling vimos que la gran mayoría de los delitos eran "de bajo impacto" o "hecho no delictivo". Quitar estos delitos nos permitiria simplificar el análisis, además de ocupar menos memoria. 

En primer lugar debemos revisar que tipo de delitos están en estas clasificaciones revisando los datos.

Para lograrlo:
* Encontrar todos los delitos que pertenezcan a las clasificaciones de interes usando _.isin()_
* Seleccionar las filas de interes y la columna de delitos con _.loc[]_
* Obtener los unicos con _.unique()_

In [15]:
cat_interes = ["Delito de bajo impacto", "Hecho no delictivo"]
del_bajo_impacto = df_vic.loc[df_vic['categoria_delito'].isin(cat_interes), "delito"]
del_bajo_impacto.unique()

array(['Narcomenudeo posesion simple', 'Robo a negocio sin violencia',
       'Violencia familiar', 'Robo de objetos',
       'Robo a casa habitacion sin violencia', 'Ddh fds',
       'Falsificacion de titulos al portador y documentos de credito publico',
       'Encubrimiento', 'Amenazas', 'Robo de documentos',
       'Robo de objetos del interior de un vehiculo', 'Abuso sexual',
       'Sustraccion de menores', 'Fraude', 'Robo de vehiculo de pedales',
       'Narcomenudeo posesión con fines de venta, comercio y suministro',
       'Abuso de confianza',
       'Lesiones culposas por transito vehicular en colision',
       'Lesiones intencionales por golpes',
       'Secuestro express (para cometer robo o extorsión)',
       'Perdida de la vida por otras causas', 'Personas extraviadas',
       'Homicidio culposo por tránsito vehicular (colision)',
       'Tentativa de suicidio', 'Robo de dinero',
       'Robo a pasajero a bordo de transporte público con violencia',
       'Robo de acce

Para ver los delitos de "alto impacto" usaremos exactamente el mismo método, solo que agregaremos una negación _~_ al selector de filas.

In [16]:
cat_interes = ["Delito de bajo impacto", "Hecho no delictivo"]
del_alto_impacto = df_vic.loc[~df_vic['categoria_delito'].isin(cat_interes), "delito"]
del_alto_impacto.unique()

array(['Robo a negocio con violencia',
       'Robo a transeunte en via publica sin violencia',
       'Robo a transeunte en via publica con violencia',
       'Robo a pasajero a bordo de metro con violencia',
       'Robo de vehiculo de servicio particular sin violencia',
       'Robo de motocicleta sin violencia',
       'Robo de vehiculo de servicio particular con violencia',
       'Lesiones intencionales por arma de fuego',
       'Homicidio por arma de fuego',
       'Robo a pasajero a bordo de metro sin violencia',
       'Robo de vehiculo de servicio público con violencia',
       'Robo a pasajero a bordo de taxi con violencia',
       'Robo de motocicleta con violencia',
       'Robo a repartidor con violencia',
       'Robo a casa habitacion con violencia',
       'Robo a transeunte saliendo del banco con violencia',
       'Homicidio por golpes',
       'Robo a pasajero a bordo de pesero colectivo con violencia',
       'Violacion', 'Robo de vehiculo de servicio oficial sin 

Quitar los delitos de bajo impacto implica que nos concentramos en los que más afectan a la población, tales como: robo, secuestro y homicidio. No obstante, al mismo tiempo implica ignorar la forma de contacto con la violencia más común dentro de la población. La decisión de ignorar datos siempre debé de tomarse tomando en cuenta las implicaciones.

En este caso, para simplificar los análisis quitaremos los delitos de bajo impacto y los hechos no delictivos. 

Con la finalidad de quitar los delitos que no nos interesan aplicaremos una modificación del comando _isin()_. En el comando anterior vimos los delitos en la lista, para seleccionar los delitos que no están en la lista agregaremos _~_ al inicio del comando, lo cual representa una negación.

In [17]:
df_vic = df_vic.loc[~df_vic['categoria_delito'].isin(['Delito de bajo impacto','Hecho no delictivo'])]

Después, de estas transformaciones nuestros datos son menos.

In [18]:
df_vic.shape

(32168, 8)

__Tipo de persona y cálidad jurídica__

Una pregunta que podemos hacernos es si hay una relación entre el tipo de persona y la calidad juridica. 

Para verificar esto nos centraremos en análizar solo esas dos filas y usaremos el comando _.drop_duplicates()_, el cual quita todas las filas repetidas. En primer lugar seleccionaremos las dos columnas que nos interesan ('TipoPersona' y 'CalidadJuridica'), para después quitar todos los duplicados y comparar si siempre coinciden.

En este ocasión no vamos a guardar el resultado, solo veremos lo que sucede sin modificar los datos, y a partir de eso tomaremos una decisión.

In [19]:
df_vic[['TipoPersona', 'CalidadJuridica']].drop_duplicates()

,TipoPersona,CalidadJuridica
12,Moral,Victima
13,Fisica,Victima y denunciante
41,Fisica,Victima
60,Fisica,Lesionado
64,Fisica,Cadaver
1247,NaN,Victima y denunciante
2153,NaN,Victima
6985,Moral,Victima y denunciante
8254,Moral,Cadaver
14580,Fisica,Fallecido


Podemos apreciar que no hay relación entre el tipo de persona y la calidad jurídica, por lo que ambas columnas nos proporcionan información relevante y dejaremos ambas. 
Este es un ejemplo de por que es importante revisar y entender los datos antes de modificarlos, ya que no siempre las transformaciones que hacemos durante el proceso de limpieza son adecuadas o necesarias.

Sin embargo, como podemos ver la Calidad Juridica tiene categorias que son equivalente como "Cadaver y Fallecido" o que podrían ser absorvidas en categorías mas grandes como "Menor víctima", "Victima  niño" y "Lesionado  adolescente". En este caso vamos a tratar de simplificar las categorias lo mas posible, en gran parte por que no todos los menores de edad vienen categorizados como "Menor víctima", lo cual puede causar errores en nuestros análisis posteriores.

In [20]:
df_vic['CalidadJuridica'].value_counts()

Victima y denunciante     22993
Victima                    7008
Cadaver                    1293
Lesionado                   860
Fallecido                     7
Menor víctima                 3
Victima  niño                 2
Lesionado  adolescente        2
Name: CalidadJuridica, dtype: int64

El comando _replace_ toma un diccionario de python para hacer una lista de remplazo. Por ejemplo, para remplazar las categorías en las que dectamos error usaremos el siguiente dicionario de remplazo. Recuerda que los diccionarios usar _:_ para unir los pares de llave:valor y _,_ para separa los pares.

In [21]:
remplazo = {"Fallecido":"Cadaver",
            "Menor víctima":"Victima",
            "Victima  niño":"Victima",
            "Lesionado  adolescente":"Lesionado"}
df_vic['CalidadJuridica'] = df_vic['CalidadJuridica'].replace(remplazo)

df_vic['CalidadJuridica'].value_counts()

Victima y denunciante    22993
Victima                   7013
Cadaver                   1300
Lesionado                  862
Name: CalidadJuridica, dtype: int64

Ahora las categorias juridicas que detectamos han sido sustituidas y llas víctimas se han sumado a sus nuevas categorias. Sin embargo, todas las mcategorias que no estaban en el diccionario de remplazo quedaron exactamente igual. Este metodo puede ser muy costoso cuando se quieren hacer muchos remplazos, ya que tendriamos que crear un diccionario con todas las opciones que deseamos remplazar. 

Una opción en esos casos es el comando _.map()_. El comando map remplaza solo los datos que están en el diccionario y llena todos aquellos que no están con NaN.

#### Guardar datos

Es posible guardar los datos limpios que hemos obtenido de varias formas, podemos guardarlos como csv o excel usando los comandos _.to_csv_ y _.to_excel_. Estos formatos tienen la ventaja de que son faciles de compartir. 

Nosotros guardaremos los datos limpios en la carpeta _data_clean_ como un csv, debido a que el tamaño de la base de datos es grande, puede ser complicado para manipularla en excel. Sin embargo, es posible abrir archivos csv usando excel.

Es muy importante no rescribir los datos originales y tratar de tener una carpeta para cada parte del proceso de análisis, para evitar perder información y poder reproducir confiablemente nuestros análisis.

In [23]:
df_vic.to_csv('./data-clean/VictimasPgjCdmx-AltoImpacto.csv', index=False)

Una desventaja de guardar los archivos usando csv o excel, es que podemos perder el formato y los tipos de datos. Esto es importante sobretodo para tipos de datos como _datetime_. Una opción es guardar nuestros datos en un formato que sea facilmente interpretable para python, aunque este no se pueda trabajar con excel.

In [25]:
from joblib import dump

with open('./data-clean/VictimasPgjCdmx-AltoImpacto.pkl', 'wb') as f:
    dump(df_vic, f)

#### Ejercicio 5

Revisa el archivo [CP-EjemploLimpiezaCarpetas](./extras/CP-EjemploLimpiezaCarpetas.ipynb) que se encuentra en la sección de extras. Este archivo realiza una limpieza similar para el conjunto de datos de las carpetas. Escribe que hace cada uno de los pasos y da las razones por las cuales se hacen.

## 3.5 Tipos de datos

Cómo vimos en la sección anterior hay varios tipos de datos como: texto, númerico, fecha, etc.
De manera automática pandas infiere el tipo de datos al leer el archivo, de una manera muy similar a como Excel interpreta si un dato es texto o numérico.

Los tipos de datos de pandas son:
* `object`: texto o mezcla de varios tipos de datos, este es el tipo de dato por defecto
* `category`: información categórica, similar a Factors en R
* `int64`: números enteros
* `float64`: números con punto decimal
* `bool`: valores verdadero o falso
* `datetime64`: fechas
* `timedelta[ns]`: diferencias de tiempo

Podemos ver los tipos de datos inferidos usando la opción `.dtypes`.

In [20]:
df.dtypes

ID                              object
Estado_origen                   object
Municipio_origen                object
Panteón_origen                  object
Estatus_FC                      object
Fecha_inhumación        datetime64[ns]
Fecha_defunción                 object
Fecha_exhumación        datetime64[ns]
Restos_tipo                     object
Sexo                            object
Edad                            object
Conocido_Desconocido            object
Primer apellido                 object
Segundo Apellido                object
Nombre(s)                       object
Nombre completo                 object
Institución_origen              object
Rdoc                            object
Marca_temporal          datetime64[ns]
Datos alternativos             float64
dtype: object

### Texto

Dependiendo del tipo de dato se pueden hacer diferentes operaciones. Por ejemplo, se puede pasar a mayusculas un texto pero no un número, se puede obtener el año de una fecha pero no de un texto

Por ejemplo, pensemos que queremos que las instituciones de origen no esten todas en mayusculas, sino solo la primera letra de la palabra.

Para hacer esto seleccionaremos la columna de interes, y luego aplicaremos la función `.str.title()`. 

**Nota**: existen varias funciones para mayusculas y minusculas como `.upper()`, `.lower()`, `.capitalize()` y `.title()`

In [48]:
df['Institución_origen'].str.title()

0            Universidad Westhill - Facultad De Medicina
1      Instituto De Ciencias Forenses - Tribunal Supe...
2      Instituto De Ciencias Forenses - Tribunal Supe...
3      Instituto De Ciencias Forenses - Tribunal Supe...
4      Instituto De Ciencias Forenses - Tribunal Supe...
                             ...                        
995    Universidad Nacional Autónoma De México - Facu...
996    Universidad Nacional Autónoma De México - Facu...
997    Instituto De Ciencias Forenses - Tribunal Supe...
998    Instituto De Ciencias Forenses - Tribunal Supe...
999    Instituto De Ciencias Forenses - Tribunal Supe...
Name: Institución_origen, Length: 1000, dtype: object

Podemos realizar más operaciones sobre esta selección, por ejemplo ordenar alfabeticamente obtener los valores únicos para obtener una lista de nombres propios.

In [49]:
df['Institución_origen'].str.title().sort_values().unique()

array(['Centro Cultural Universitario Justo Sierra',
       'Escuela De Medicina Saint Luke',
       'Institución De Asistencia Privada - Escuela Libre De Homeopatía De México',
       'Instituto De Ciencias Forenses - Tribunal Superior De Justicia De La Ciudad De México',
       'Instituto Politécnico Nacional - Escuela Nacional De Medicina Y Homeopatía',
       'Procuraduría General De La República',
       'Secretaría De La Defensa Nacional - Escuela Militar De Medicina',
       'The American British Cowdray Medical Center La.P.',
       'Universidad Anáhuac - Facultad De Ciencias De La Salud',
       'Universidad Nacional Autónoma De México - Facultad De Medicina',
       'Universidad Popular Autónoma Del Estado De Puebla',
       'Universidad Tominaga Nakamoto S.C. Escuela De Medicina Ciencias Básicas',
       'Universidad Westhill - Facultad De Medicina'], dtype=object)

Existen funciones que no se encuentran en pandas pero que pueden ser utiles. Por ejemplo, para ejemploquitar los acentos es posible con la función `unidecode`, la cual es parte de la biblioteca `unidecode`.

In [50]:
from unidecode import unidecode
unidecode('México')

'Mexico'

Podemos aplicar esta función a toda la columna

In [51]:
df['Institución_origen'].apply(unidecode)

0            UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA
1      INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
2      INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
3      INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
4      INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
                             ...                        
995    UNIVERSIDAD NACIONAL AUTONOMA DE MEXICO - FACU...
996    UNIVERSIDAD NACIONAL AUTONOMA DE MEXICO - FACU...
997    INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
998    INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
999    INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
Name: Institución_origen, Length: 1000, dtype: object

Además, podemos aplicar varias funciones juntas.

In [53]:
df['Institución_origen'].str.title().apply(unidecode)

0            Universidad Westhill - Facultad De Medicina
1      Instituto De Ciencias Forenses - Tribunal Supe...
2      Instituto De Ciencias Forenses - Tribunal Supe...
3      Instituto De Ciencias Forenses - Tribunal Supe...
4      Instituto De Ciencias Forenses - Tribunal Supe...
                             ...                        
995    Universidad Nacional Autonoma De Mexico - Facu...
996    Universidad Nacional Autonoma De Mexico - Facu...
997    Instituto De Ciencias Forenses - Tribunal Supe...
998    Instituto De Ciencias Forenses - Tribunal Supe...
999    Instituto De Ciencias Forenses - Tribunal Supe...
Name: Institución_origen, Length: 1000, dtype: object

Sin embargo, al aplicar esta función a la columna `Nombre(s)` genera un error `AttributeError: 'float' object has no attribute 'encode'`. 

Esto se debe a que la columna incluye valores vacios, los cuales son de tipo `float`, por lo tanto la función `unidecode` se confunde, ya que los números no tienen acentos. 

Resolver este tipo de problemas es parte de la limpieza de datos y se verá mas adelante.

In [54]:
df['Nombre(s)'].apply( unidecode )

AttributeError: 'float' object has no attribute 'encode'

Dejo aqui una posible solución sin explicación (por el momento).

In [60]:
df['Nombre(s)'].apply( lambda s: unidecode(s) if type(s)==str  else s )

0              NaN
1              NaN
2          Alberto
3              NaN
4              NaN
          ...     
995         Sergio
996    Juan Carlos
997            NaN
998            NaN
999            NaN
Name: Nombre(s), Length: 1000, dtype: object

### Categoria

En general se considera que un dato es categorico cuando hay pocas posibilidades definidas. Una aproximación rápida es que tenga menos de diez opciones de texto. Sin embargo esta no es una regla dura, por ejemplo el catálogo del INEGI de Estados, Municipios y Localidades tiene miles de opciones.

Podemos convertir una columna de tipo `object` a `category` con la función `.astype("category")`. 

En este ejemplo caso reescribiremos la columna para poder realizar varias operaciones.

In [67]:
df['Restos_tipo'] = df['Restos_tipo'].astype("category")
df['Restos_tipo']

0      Restos cremados
1              Cadáver
2              Cadáver
3              Cadáver
4              Cadáver
            ...       
995            Cadáver
996            Cadáver
997            Cadáver
998            Cadáver
999            Cadáver
Name: Restos_tipo, Length: 1000, dtype: category
Categories (6, object): ['Cadáver', 'Feto', 'Miembros', 'Restos cremados', 'Restos humanos', 'Restos óseos']

Podemos ver que el tipo a cambiado

In [68]:
df.dtypes

ID                              object
Estado_origen                   object
Municipio_origen                object
Panteón_origen                  object
Estatus_FC                      object
Fecha_inhumación        datetime64[ns]
Fecha_defunción                 object
Fecha_exhumación        datetime64[ns]
Restos_tipo                   category
Sexo                            object
Edad                            object
Conocido_Desconocido            object
Primer apellido                 object
Segundo Apellido                object
Nombre(s)                       object
Nombre completo                 object
Institución_origen              object
Rdoc                            object
Marca_temporal          datetime64[ns]
Datos alternativos             float64
dtype: object

Una ventaja de usar `category` es que permite definir un orden diferente al alfabético o numérico. Por ejemplo, definamos un orden especial para el tipo de restos.

In [72]:
from pandas.api.types import CategoricalDtype

restos_categorias = 'Feto', 'Restos humanos', 'Miembros', 'Restos óseos', 'Restos cremados', 'Cadáver'
restos_categorias = CategoricalDtype(categories=restos_categorias, ordered=True)
df['Restos_tipo'] = df['Restos_tipo'].astype( restos_categorias )
df['Restos_tipo']

0      Restos cremados
1              Cadáver
2              Cadáver
3              Cadáver
4              Cadáver
            ...       
995            Cadáver
996            Cadáver
997            Cadáver
998            Cadáver
999            Cadáver
Name: Restos_tipo, Length: 1000, dtype: category
Categories (6, object): ['Feto' < 'Restos humanos' < 'Miembros' < 'Restos óseos' < 'Restos cremados' < 'Cadáver']

Ahora si ordenamos por categoria apareceran primero los Fetos y al final los Cadáveres.

In [73]:
df.sort_values(by='Restos_tipo')

,ID,Estado_origen,Municipio_origen,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Rdoc,Marca_temporal,Datos alternativos
492,XX-P493,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-02-02,2019-01-12 00:00:00,NaT,Feto,Femenino,14 semanas,Conocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-30 12:34:48.000,NaN
414,XX-P415,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-05-25,2019-04-28 00:00:00,NaT,Feto,Masculino,NaN,Conocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-30 10:48:58.000,NaN
795,XX-P796,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2017-10-07,S/D,NaT,Feto,Masculino,18 semanas,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-01-04 17:19:03.000,NaN
455,XX-P456,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-05-04,2019-04-05 00:00:00,NaT,Feto,Indeterminado,14 semanas,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-30 11:41:11.000,NaN
485,XX-P486,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-02-02,2019-01-03 00:00:00,NaT,Feto,Masculino,13 semanas,Conocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-30 12:26:44.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,XX-P242,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2018-10-14,2017-12-18 00:00:00,NaT,Cadáver,Masculino,54,Conocido,Peña,Padilla,Alejandro,Alejandro Peña Padilla,UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO - FACU...,Sí,2020-03-24 00:00:00.000,NaN
240,XX-P241,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2018-10-14,2017-12-17 00:00:00,NaT,Cadáver,Masculino,45,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO - FACU...,Sí,2020-03-24 00:00:00.000,NaN
239,XX-P240,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2018-10-14,2017-12-14 00:00:00,NaT,Cadáver,Masculino,60,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO - FACU...,Sí,2020-03-24 00:00:00.000,NaN
211,XX-P212,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2018-10-27,2018-09-19 00:00:00,NaT,Cadáver,Masculino,NaN,Conocido,Vargas,Rosas,Jesús,Jesús Vargas Rosas,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-24 00:00:00.000,NaN


### Números

Existen dos tipos de datos númericos, enteros y flotantes. Los dos se comportan de forma muy similar.

En este conjunto de datos la columna `Edad` contiene numeros, sin embargo su tipo es `object` ya que hay edades como "18 semanas" que corresponden a fetos y neonatos.

In [26]:
df['Edad'].str.upper()

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
995    NaN
996    NaN
997    NaN
998    NaN
999    NaN
Name: Edad, Length: 1000, dtype: object

In [22]:
df['Fecha_inhumación'].dt.year

0      2019
1      2019
2      2019
3      2019
4      2019
       ... 
995    2018
996    2018
997    2018
998    2018
999    2018
Name: Fecha_inhumación, Length: 1000, dtype: int64

En caso de duda pandas asigna el tipo de dato object e infiere celda por celda el tipo de dato específico.

In [21]:
df['Datos alternativos'].unique()

array([nan])

In [ ]:

1. Escribe tres formas en las que te gustaría ordenar los datos.
2. Obten las tablas ordenadas, en caso de no poder hacerlo indica que datos te harían falta.